# 🐍 Session 13 : SQL depuis Python

## Objectifs de cette session
- Connecter Python à une base de données SQLite
- Exécuter des requêtes SQL depuis Python
- Utiliser pandas pour analyser les résultats
- Exporter les données vers différents formats

---

## 1. Configuration et Imports

Commençons par importer les bibliothèques nécessaires.

In [ ]:
# Imports standard
import sqlite3
import os
from pathlib import Path

# Imports pour analyse de données
import pandas as pd
import numpy as np

# SQLAlchemy pour connexion base de données
from sqlalchemy import create_engine

# Configuration pandas pour meilleur affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports réussis")
print(f"Version pandas : {pd.__version__}")
print(f"Version numpy : {np.__version__}")

## 2. Connexion à la Base de Données

Nous allons nous connecter à la base `sales.db` créée dans les sessions précédentes.

In [ ]:
# Chemin vers la base de données
db_path = '../sql/sales.db'

# Vérifier que la base existe
if not os.path.exists(db_path):
    print(f"❌ Erreur : La base de données {db_path} n'existe pas.")
    print("Veuillez d'abord créer la base avec le script schema_sales.sql")
else:
    print(f"✅ Base de données trouvée : {db_path}")
    print(f"Taille : {os.path.getsize(db_path) / 1024:.2f} KB")

# Créer une connexion SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}')

print("\n✅ Connexion SQLAlchemy créée avec succès")

### 2.1 Vérifier les tables disponibles

Avant de commencer, vérifions quelles tables sont disponibles dans notre base.

In [ ]:
# Récupérer la liste des tables
query_tables = """
SELECT name 
FROM sqlite_master 
WHERE type='table'
ORDER BY name;
"""

tables = pd.read_sql_query(query_tables, engine)
print("📊 Tables disponibles dans la base de données :")
print(tables)

# Compter le nombre d'enregistrements dans chaque table
print("\n📈 Nombre d'enregistrements par table :")
for table in tables['name']:
    count_query = f"SELECT COUNT(*) as count FROM {table}"
    count = pd.read_sql_query(count_query, engine)['count'][0]
    print(f"  {table:20} : {count:5} enregistrements")

## 3. Requêtes SELECT Simples

### 3.1 Récupérer tous les clients

In [ ]:
# Requête simple : récupérer tous les clients
query_customers = """
SELECT 
    customer_id,
    first_name,
    last_name,
    email,
    country,
    city
FROM customers
ORDER BY last_name, first_name
LIMIT 10;
"""

df_customers = pd.read_sql_query(query_customers, engine)

print("👥 Les 10 premiers clients :")
print(df_customers)

print(f"\n📊 Forme du DataFrame : {df_customers.shape}")
print(f"Colonnes : {list(df_customers.columns)}")

### 3.2 Statistiques sur les clients

In [ ]:
# Récupérer tous les clients pour faire des statistiques
df_all_customers = pd.read_sql_query("SELECT * FROM customers", engine)

print("📊 Statistiques sur les clients :")
print(f"Nombre total de clients : {len(df_all_customers)}")
print(f"\nRépartition par pays :")
print(df_all_customers['country'].value_counts())

print(f"\nRépartition par ville (top 10) :")
print(df_all_customers['city'].value_counts().head(10))

### 3.3 Filtrage avec paramètres (Sécurité)

⚠️ **Important** : Toujours utiliser des paramètres pour éviter les injections SQL !

In [ ]:
# ✅ BONNE PRATIQUE : Utiliser des paramètres
country_filter = 'France'

query_filtered = """
SELECT 
    customer_id,
    first_name,
    last_name,
    email,
    city
FROM customers
WHERE country = :country
ORDER BY city, last_name;
"""

df_france = pd.read_sql_query(
    query_filtered, 
    engine, 
    params={'country': country_filter}
)

print(f"👥 Clients en {country_filter} :")
print(df_france.head(10))
print(f"\n📊 Total : {len(df_france)} clients")

## 4. Récupération des Produits

Analysons le catalogue de produits.

In [ ]:
# Récupérer tous les produits
query_products = """
SELECT 
    product_id,
    product_name,
    category,
    price,
    stock_quantity
FROM products
ORDER BY category, product_name;
"""

df_products = pd.read_sql_query(query_products, engine)

print("🛍️ Catalogue de produits :")
print(df_products)

# Statistiques sur les produits
print("\n📊 Statistiques :")
print(f"Nombre total de produits : {len(df_products)}")
print(f"\nProduits par catégorie :")
print(df_products['category'].value_counts())
print(f"\nPrix moyen par catégorie :")
print(df_products.groupby('category')['price'].agg(['mean', 'min', 'max']).round(2))
print(f"\nStock total par catégorie :")
print(df_products.groupby('category')['stock_quantity'].sum())

### 4.1 Produits à faible stock

Identifions les produits qui nécessitent un réapprovisionnement.

In [ ]:
# Produits avec stock inférieur à 20
low_stock_threshold = 20

query_low_stock = """
SELECT 
    product_id,
    product_name,
    category,
    price,
    stock_quantity
FROM products
WHERE stock_quantity < :threshold
ORDER BY stock_quantity ASC, product_name;
"""

df_low_stock = pd.read_sql_query(
    query_low_stock, 
    engine, 
    params={'threshold': low_stock_threshold}
)

print(f"⚠️ Produits avec stock < {low_stock_threshold} :")
if len(df_low_stock) > 0:
    print(df_low_stock)
else:
    print("✅ Aucun produit en rupture de stock")

## 5. Requêtes JOIN : Commandes et Clients

### 5.1 Vue d'ensemble des commandes

In [ ]:
# JOIN entre orders et customers
query_orders = """
SELECT 
    o.order_id,
    o.order_date,
    o.total_amount,
    o.status,
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.country,
    c.city
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
ORDER BY o.order_date DESC;
"""

df_orders = pd.read_sql_query(query_orders, engine)

print("🛒 Commandes avec informations clients :")
print(df_orders.head(10))

print(f"\n📊 Statistiques générales :")
print(f"Nombre total de commandes : {len(df_orders)}")
print(f"Montant total des ventes : {df_orders['total_amount'].sum():.2f} €")
print(f"Panier moyen : {df_orders['total_amount'].mean():.2f} €")
print(f"Panier médian : {df_orders['total_amount'].median():.2f} €")

### 5.2 Analyse par statut de commande

In [ ]:
# Analyse par statut
print("📊 Analyse par statut de commande :")
status_analysis = df_orders.groupby('status').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean', 'median']
}).round(2)

status_analysis.columns = ['Nombre', 'Total (€)', 'Moyenne (€)', 'Médiane (€)']
print(status_analysis)

# Distribution des statuts
print("\n📈 Distribution des statuts :")
print(df_orders['status'].value_counts())

### 5.3 Ventes par pays

In [ ]:
# Analyse par pays
print("🌍 Ventes par pays :")
country_sales = df_orders.groupby('country').agg({
    'order_id': 'count',
    'customer_id': 'nunique',
    'total_amount': ['sum', 'mean']
}).round(2)

country_sales.columns = ['Nb Commandes', 'Nb Clients', 'CA Total (€)', 'Panier Moyen (€)']
country_sales = country_sales.sort_values('CA Total (€)', ascending=False)
print(country_sales)

### 5.4 Top 10 clients (par montant total)

In [ ]:
# Top clients
query_top_customers = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.country,
    COUNT(o.order_id) as nb_orders,
    SUM(o.total_amount) as total_spent,
    AVG(o.total_amount) as avg_order,
    MAX(o.order_date) as last_order_date
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.email, c.country
ORDER BY total_spent DESC
LIMIT 10;
"""

df_top_customers = pd.read_sql_query(query_top_customers, engine)

print("🏆 Top 10 clients (par montant dépensé) :")
print(df_top_customers)

## 6. Analyse Détaillée : Produits Vendus

### 6.1 JOIN complexe sur 4 tables

In [ ]:
# Requête complexe : toutes les informations sur les ventes
query_sales_detail = """
SELECT 
    o.order_id,
    o.order_date,
    o.status,
    c.customer_id,
    c.first_name || ' ' || c.last_name as customer_name,
    c.country,
    c.city,
    p.product_id,
    p.product_name,
    p.category,
    oi.quantity,
    oi.unit_price,
    (oi.quantity * oi.unit_price) as line_total
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_date DESC, o.order_id, p.product_name;
"""

df_sales = pd.read_sql_query(query_sales_detail, engine)

print("💰 Détail complet des ventes :")
print(df_sales.head(15))

print(f"\n📊 Résumé :")
print(f"Nombre de lignes de commande : {len(df_sales)}")
print(f"Nombre de commandes uniques : {df_sales['order_id'].nunique()}")
print(f"Nombre de produits différents vendus : {df_sales['product_id'].nunique()}")
print(f"Montant total des ventes : {df_sales['line_total'].sum():.2f} €")

### 6.2 Top 10 produits les plus vendus

In [ ]:
# Analyser les produits les plus vendus
print("🏆 Top 10 produits par quantité vendue :")
top_products_qty = df_sales.groupby(['product_id', 'product_name', 'category']).agg({
    'quantity': 'sum',
    'line_total': 'sum'
}).round(2)

top_products_qty = top_products_qty.sort_values('quantity', ascending=False).head(10)
top_products_qty.columns = ['Quantité Vendue', 'CA Total (€)']
print(top_products_qty)

print("\n🏆 Top 10 produits par chiffre d'affaires :")
top_products_revenue = df_sales.groupby(['product_id', 'product_name', 'category']).agg({
    'quantity': 'sum',
    'line_total': 'sum'
}).round(2)

top_products_revenue = top_products_revenue.sort_values('line_total', ascending=False).head(10)
top_products_revenue.columns = ['Quantité Vendue', 'CA Total (€)']
print(top_products_revenue)

### 6.3 Ventes par catégorie

In [ ]:
# Analyse par catégorie de produits
print("📦 Ventes par catégorie :")
category_sales = df_sales.groupby('category').agg({
    'order_id': 'count',
    'quantity': 'sum',
    'line_total': 'sum'
}).round(2)

category_sales.columns = ['Nb Lignes', 'Quantité Totale', 'CA Total (€)']
category_sales = category_sales.sort_values('CA Total (€)', ascending=False)
print(category_sales)

# Pourcentage par catégorie
print("\n📊 Part de marché par catégorie :")
category_sales['Part (%)'] = (category_sales['CA Total (€)'] / category_sales['CA Total (€)'].sum() * 100).round(2)
print(category_sales[['CA Total (€)', 'Part (%)']])

## 7. Analyse Temporelle

### 7.1 Conversion des dates et extraction des composantes

In [ ]:
# Créer une copie du DataFrame pour l'analyse temporelle
df_time_analysis = df_sales.copy()

# Convertir order_date en datetime
df_time_analysis['order_date'] = pd.to_datetime(df_time_analysis['order_date'])

# Extraire les composantes temporelles
df_time_analysis['year'] = df_time_analysis['order_date'].dt.year
df_time_analysis['month'] = df_time_analysis['order_date'].dt.month
df_time_analysis['month_name'] = df_time_analysis['order_date'].dt.strftime('%B')
df_time_analysis['quarter'] = df_time_analysis['order_date'].dt.quarter
df_time_analysis['day_of_week'] = df_time_analysis['order_date'].dt.day_name()

print("📅 Données avec composantes temporelles :")
print(df_time_analysis[['order_date', 'year', 'month', 'month_name', 'quarter', 'day_of_week', 'line_total']].head(10))

### 7.2 Ventes par mois

In [ ]:
# Analyse mensuelle
print("📆 Ventes par mois :")
monthly_sales = df_time_analysis.groupby(['year', 'month', 'month_name']).agg({
    'order_id': 'nunique',
    'line_total': 'sum'
}).round(2)

monthly_sales.columns = ['Nb Commandes', 'CA Total (€)']
print(monthly_sales)

### 7.3 Ventes par jour de la semaine

In [ ]:
# Analyse par jour de la semaine
print("📊 Ventes par jour de la semaine :")
weekday_sales = df_time_analysis.groupby('day_of_week').agg({
    'order_id': 'nunique',
    'line_total': ['sum', 'mean']
}).round(2)

weekday_sales.columns = ['Nb Commandes', 'CA Total (€)', 'CA Moyen (€)']

# Ordre des jours de la semaine
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_sales = weekday_sales.reindex([day for day in days_order if day in weekday_sales.index])

print(weekday_sales)

## 8. Clients Inactifs (LEFT JOIN)

Identifions les clients qui n'ont jamais passé de commande.

In [ ]:
# LEFT JOIN pour trouver les clients sans commande
query_inactive = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.country,
    c.created_at,
    COUNT(o.order_id) as nb_orders
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.email, c.country, c.created_at
HAVING COUNT(o.order_id) = 0
ORDER BY c.created_at DESC;
"""

df_inactive = pd.read_sql_query(query_inactive, engine)

print("😴 Clients inactifs (sans commande) :")
if len(df_inactive) > 0:
    print(df_inactive)
    print(f"\n📊 Total : {len(df_inactive)} clients inactifs")
else:
    print("✅ Tous les clients ont passé au moins une commande !")

## 9. Création d'une Table d'Analyse

### 9.1 Créer un DataFrame de statistiques clients

In [ ]:
# Créer une table d'analyse avec statistiques par client
query_customer_stats = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    c.country,
    c.city,
    COUNT(o.order_id) as nb_orders,
    COALESCE(SUM(o.total_amount), 0) as total_spent,
    COALESCE(AVG(o.total_amount), 0) as avg_order,
    COALESCE(MIN(o.total_amount), 0) as min_order,
    COALESCE(MAX(o.total_amount), 0) as max_order,
    MAX(o.order_date) as last_order_date,
    MIN(o.order_date) as first_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.email, c.country, c.city
ORDER BY total_spent DESC;
"""

df_customer_stats = pd.read_sql_query(query_customer_stats, engine)

# Catégoriser les clients selon leur activité
def categorize_customer(row):
    if row['nb_orders'] == 0:
        return 'Inactif'
    elif row['nb_orders'] == 1:
        return 'Nouveau'
    elif row['total_spent'] < 200:
        return 'Occasionnel'
    elif row['total_spent'] < 500:
        return 'Régulier'
    else:
        return 'VIP'

df_customer_stats['segment'] = df_customer_stats.apply(categorize_customer, axis=1)

print("📊 Statistiques par client (top 15) :")
print(df_customer_stats.head(15))

print("\n📈 Répartition par segment :")
print(df_customer_stats['segment'].value_counts())

### 9.2 Sauvegarder dans une nouvelle table SQL

In [ ]:
# Sauvegarder les statistiques dans une nouvelle table
table_name = 'customer_statistics'

try:
    # Écrire le DataFrame dans la base de données
    df_customer_stats.to_sql(
        table_name, 
        engine, 
        if_exists='replace',  # Remplacer si existe
        index=False
    )
    
    print(f"✅ Table '{table_name}' créée avec succès")
    print(f"   Nombre d'enregistrements : {len(df_customer_stats)}")
    
    # Vérifier que la table a été créée
    verify_query = f"SELECT COUNT(*) as count FROM {table_name}"
    count = pd.read_sql_query(verify_query, engine)['count'][0]
    print(f"   Vérification : {count} lignes dans la table")
    
except Exception as e:
    print(f"❌ Erreur lors de la création de la table : {e}")

## 10. Export des Données

### 10.1 Export vers CSV

In [ ]:
# Créer un dossier pour les exports
export_dir = Path('../data/exports')
export_dir.mkdir(parents=True, exist_ok=True)

# Export des statistiques clients
csv_file = export_dir / 'customer_statistics.csv'
df_customer_stats.to_csv(csv_file, index=False, encoding='utf-8')
print(f"✅ Export CSV : {csv_file}")
print(f"   Taille : {csv_file.stat().st_size / 1024:.2f} KB")

# Export du détail des ventes
csv_sales = export_dir / 'sales_detail.csv'
df_sales.to_csv(csv_sales, index=False, encoding='utf-8')
print(f"✅ Export CSV : {csv_sales}")
print(f"   Taille : {csv_sales.stat().st_size / 1024:.2f} KB")

### 10.2 Export vers JSON

In [ ]:
# Export JSON du top 10 clients
json_file = export_dir / 'top_customers.json'
df_top_customers.to_json(json_file, orient='records', indent=2)
print(f"✅ Export JSON : {json_file}")
print(f"   Taille : {json_file.stat().st_size / 1024:.2f} KB")

### 10.3 Créer un rapport HTML

In [ ]:
# Créer un rapport HTML simple
html_report = export_dir / 'sales_report.html'

html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Rapport de Ventes - Session 13</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1 {{ color: #2c3e50; }}
        h2 {{ color: #34495e; border-bottom: 2px solid #3498db; padding-bottom: 5px; }}
        table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
        th {{ background-color: #3498db; color: white; padding: 10px; text-align: left; }}
        td {{ padding: 8px; border-bottom: 1px solid #ddd; }}
        tr:hover {{ background-color: #f5f5f5; }}
        .stat-box {{ background: #ecf0f1; padding: 15px; margin: 10px 0; border-radius: 5px; }}
    </style>
</head>
<body>
    <h1>📊 Rapport d'Analyse des Ventes</h1>
    <p><strong>Généré le :</strong> {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    
    <div class="stat-box">
        <h2>📈 Statistiques Globales</h2>
        <ul>
            <li><strong>Nombre de clients :</strong> {len(df_all_customers)}</li>
            <li><strong>Nombre de commandes :</strong> {len(df_orders)}</li>
            <li><strong>Chiffre d'affaires total :</strong> {df_orders['total_amount'].sum():.2f} €</li>
            <li><strong>Panier moyen :</strong> {df_orders['total_amount'].mean():.2f} €</li>
        </ul>
    </div>
    
    <h2>🏆 Top 10 Clients</h2>
    {df_top_customers.to_html(index=False)}
    
    <h2>📦 Ventes par Catégorie</h2>
    {category_sales.to_html()}
    
    <h2>🌍 Ventes par Pays</h2>
    {country_sales.to_html()}
</body>
</html>
"""

with open(html_report, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✅ Rapport HTML créé : {html_report}")
print(f"   Ouvrez le fichier dans votre navigateur pour voir le rapport")

## 11. Fonctions Réutilisables

Créons des fonctions pour faciliter les requêtes futures.

In [ ]:
def get_customers_by_country(country_name, engine):
    """
    Récupère tous les clients d'un pays donné.
    
    Args:
        country_name (str): Nom du pays
        engine: Moteur SQLAlchemy
        
    Returns:
        DataFrame: Clients du pays spécifié
    """
    query = """
    SELECT * FROM customers
    WHERE country = :country
    ORDER BY last_name, first_name
    """
    
    try:
        df = pd.read_sql_query(query, engine, params={'country': country_name})
        return df
    except Exception as e:
        print(f"Erreur : {e}")
        return None


def get_order_summary(start_date=None, end_date=None, engine=engine):
    """
    Récupère un résumé des commandes sur une période.
    
    Args:
        start_date (str): Date de début (YYYY-MM-DD)
        end_date (str): Date de fin (YYYY-MM-DD)
        engine: Moteur SQLAlchemy
        
    Returns:
        DataFrame: Résumé des commandes
    """
    query = "SELECT * FROM orders WHERE 1=1"
    params = {}
    
    if start_date:
        query += " AND order_date >= :start_date"
        params['start_date'] = start_date
    
    if end_date:
        query += " AND order_date <= :end_date"
        params['end_date'] = end_date
    
    query += " ORDER BY order_date DESC"
    
    try:
        df = pd.read_sql_query(query, engine, params=params)
        return df
    except Exception as e:
        print(f"Erreur : {e}")
        return None


def get_product_performance(engine):
    """
    Analyse la performance des produits.
    
    Returns:
        DataFrame: Statistiques par produit
    """
    query = """
    SELECT 
        p.product_id,
        p.product_name,
        p.category,
        p.price as current_price,
        COUNT(DISTINCT oi.order_id) as nb_orders,
        SUM(oi.quantity) as total_quantity_sold,
        SUM(oi.quantity * oi.unit_price) as total_revenue,
        AVG(oi.unit_price) as avg_selling_price
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.product_id, p.product_name, p.category, p.price
    ORDER BY total_revenue DESC;
    """
    
    try:
        df = pd.read_sql_query(query, engine)
        return df
    except Exception as e:
        print(f"Erreur : {e}")
        return None

print("✅ Fonctions créées avec succès")
print("   - get_customers_by_country()")
print("   - get_order_summary()")
print("   - get_product_performance()")

### 11.1 Tester les fonctions

In [ ]:
# Test : Clients en France
print("Test 1 : Clients en France")
df_france = get_customers_by_country('France', engine)
if df_france is not None:
    print(f"Nombre de clients français : {len(df_france)}")
    print(df_france.head())

print("\n" + "="*80 + "\n")

# Test : Performance des produits
print("Test 2 : Performance des produits")
df_product_perf = get_product_performance(engine)
if df_product_perf is not None:
    print(df_product_perf.head(10))

print("\n" + "="*80 + "\n")

# Test : Commandes en 2024
print("Test 3 : Commandes en 2024")
df_2024 = get_order_summary(start_date='2024-01-01', end_date='2024-12-31', engine=engine)
if df_2024 is not None:
    print(f"Nombre de commandes en 2024 : {len(df_2024)}")
    print(f"CA total 2024 : {df_2024['total_amount'].sum():.2f} €")

## 12. Résumé et Bonnes Pratiques

### ✅ Ce que nous avons appris

1. **Connexion à SQLite** avec `sqlite3` et `SQLAlchemy`
2. **Exécution de requêtes** SELECT avec `pandas.read_sql_query()`
3. **Requêtes paramétrées** pour la sécurité (éviter les injections SQL)
4. **JOIN complexes** sur plusieurs tables
5. **Analyse de données** avec pandas après extraction SQL
6. **Création de tables** avec `to_sql()`
7. **Export de données** vers CSV, JSON, HTML
8. **Fonctions réutilisables** pour faciliter les requêtes

### 🎯 Bonnes Pratiques

#### Sécurité
- ✅ Toujours utiliser des **paramètres** dans les requêtes
- ❌ Ne jamais concaténer des variables dans le SQL

#### Performance
- ✅ Privilégier **une requête JOIN** plutôt que plusieurs requêtes
- ✅ Utiliser **LIMIT** quand vous testez
- ✅ Créer des **index** sur les colonnes utilisées dans WHERE/JOIN

#### Organisation
- ✅ Fermer les connexions (ou utiliser context managers)
- ✅ Gérer les erreurs avec **try/except**
- ✅ Créer des **fonctions réutilisables**
- ✅ Documenter le code avec des **docstrings**

#### Analyse
- ✅ Utiliser **pandas** pour transformer et analyser
- ✅ Convertir les dates avec `pd.to_datetime()`
- ✅ Utiliser `groupby()` pour les agrégations

### 📚 Ressources

- [Documentation pandas.read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)
- [Documentation SQLAlchemy](https://docs.sqlalchemy.org/)
- [Python sqlite3](https://docs.python.org/3/library/sqlite3.html)

## 🎓 Exercices à Faire

### Exercice 1 : Analyse RFM (Recency, Frequency, Monetary)

Créez une analyse RFM des clients :
- **R** (Recency) : Nombre de jours depuis la dernière commande
- **F** (Frequency) : Nombre de commandes
- **M** (Monetary) : Montant total dépensé

Segmentez les clients en fonction de ces critères.

### Exercice 2 : Rapport de Ventes Mensuel

Créez un rapport qui affiche pour chaque mois :
- Nombre de commandes
- Chiffre d'affaires
- Nombre de nouveaux clients
- Panier moyen

### Exercice 3 : Analyse de Panier

Identifiez les produits fréquemment achetés ensemble (market basket analysis).

### Exercice 4 : Dashboard de Ventes

Créez un rapport HTML interactif avec :
- KPIs principaux
- Tableaux de synthèse
- Top produits/clients
- Analyse temporelle

---

**Bravo !** Vous maîtrisez maintenant l'intégration de SQL et Python ! 🎉